# Making tiddlers for TiddlyWiki from info in digiKam's SQLite database

## Format of a tiddler file

### .tid files

Looks like .tid files have some properties as key-value pairs, followed by the wikitext content of the text property. Like: 

```
created: 20191113202135473
creator: Chris
modified: 20191114214534251
modifier: Chris
title: Ni!


Linking images is important. Can't store everything in a tiddlywiki.

Timelines and calendar views.......

Inventory of stuff?

[[Link to local video file]] (text link, no thumbnail)
(etc)
```

I need to write `created`, (maybe) `modified`, and `title` fields, and then the `text`, at minimum.

### .json files

If I export multiple tiddlers to a .json file from a running TW, it (unsurprisingly) generates an array of JSON objects; each object being a tiddler. If I export one, it generates an array with one object in it:

```
[
    {
        "created": "20200217220140442",
        "text": "This tiddler has this text",
        "title": "A tiddler I made",
        "tags": "[[tag with spaces in it]] sampletag2 taggy",
        "modified": "20200217220655739",
        "custom_field": "this is the value of my custom field"
    }
]
```



Browsing around with sqlitebrowser, I locate some things I may want to read from the database.

| Thing | Table1:Column Name | Table2:Column Name (if any) |
|:---|:---|:---|
| Name of a tag | Tags:`name` |  |
| Integer id of a tag | Tags:`id` | ImageTags:`tagid` | 
| Integer id of an image | ImageTags:`imageid` | Images:`id` |
| Filename of an image | Images:`name` |  |
| Integer id of an album | Images:`album` | Albums:`id` |
| Relative path of an album within the root album | Albums:`relativePath` |
| Integer id of the root album an album belongs to | Albums:`albumRoot` | AlbumRoots:`id` |
| Path on disk of the root album | AlbumRoots:`specificPath` |
| Comment | ImageComments : `comment` |
| File creation date | ImageInformation : `creationDate` |

In [1]:
import sqlite3
import pathlib
import json

# TW wants double quotes in the JSON tiddler objects and Python's converting my dict to single quotes.
# Solution: https://stackoverflow.com/a/18283904 (could just call json.dumps() instead of defining a class for it)
class doubleQuoteDict(dict):
    def __str__(self):
        return json.dumps(self)

    def __repr__(self):
        return json.dumps(self)

In [2]:
conn = sqlite3.connect('file:digikam4.db?mode=ro', uri=True)
curs = conn.cursor()

In [3]:
def get_file_path(imageid_tuple):
    # Get the indices needed to retrieve strings we can build a path with:
    # Because I want the album id as a tuple to use as an argument, I'll make an individual search for it.
    album_id = curs.execute('SELECT album from Images WHERE id == ?', imageid_tuple).fetchone()
    if album_id == 'None':
        print('No album id: database probably not updated since image deleted.')
        return 'None'
    else:
        albumRoot_id = curs.execute('SELECT albumRoot from Albums WHERE id == ?', album_id).fetchone()
        # To build up a complete path starting from the root dir of the collection, need AlbumRoots:specificPath+Albums:relativePath+Images:name
        # For these, I want strings, not tuples, so it would be nice for the output to come like that:
        curs.row_factory = lambda cursor, row: str(row[0])
        specificPath = curs.execute('SELECT specificPath from AlbumRoots WHERE id == ?', albumRoot_id).fetchone()
        relativePath = curs.execute('SELECT relativePath from Albums WHERE id == ?', album_id).fetchone()
        f_name = curs.execute('SELECT name from Images WHERE id == ?', imageid_tuple).fetchone()
        curs.row_factory = None
        return (f_name, 'file://'+the_disk+specificPath+relativePath+'/'+f_name)

In [4]:
def get_image_info(imageid_tuple):
    res_tuple = curs.execute("SELECT orientation, width, height, format from ImageInformation WHERE imageid == ?", imageid_tuple).fetchone()
    # format is a string like 'JPG', 'RAW-NEF', 'AVI' etc
    res_str = tuple([ str(i) for i in res_tuple ])
    return res_str   #  '(orientation, width, height, file_type)'

In [5]:
def get_file_date(imageid_tuple):
    # To construct a tiddler filename/title, I might want some date info.
    # For now choose creation date, which is in the ImageInformation table.
    creation_date = str(curs.execute("SELECT creationDate from ImageInformation WHERE imageid == ?", imageid_tuple).fetchone()[0])
    d = datetime.fromisoformat(creation_date)
    return d.strftime('%Y-%m-%dT%H:%M:%S')

In [6]:
def get_caption(imageid_tuple):
    caption = curs.execute("SELECT comment from ImageComments WHERE imageid == ?", imageid_tuple).fetchone()
    if caption: 
        return str(caption[0])
    else:
        return ''

In [7]:
def get_tags(imageid_tuple):
    # If I got the images using tags, this list can't be empty
    tag_names = []
    tag_ids = curs.execute("SELECT tagid from ImageTags WHERE imageid == ?", imageid_tuple).fetchall()
    curs.row_factory = lambda cursor, row: row[0]
    for tag in tag_ids:
        tag_names.append(curs.execute("SELECT name from Tags WHERE id == ?", tag).fetchone()) 
    curs.row_factory = None
    return format_tag_list(tag_names)


In [8]:
def format_tag_list(tag_names):
    # TW tag property separates tags by spaces. Tags with spaces in them are surrounded by double square brackets
    tw_tag_string = ''
    for name in tag_names:
        if ' ' in name:
            name = '[['+name+']]'
        tw_tag_string += ' '+name
    tw_tag_string += ' [[Digikam Image]]'
    return(tw_tag_string)

In [19]:
def output_as_tid_file():
    tiddler_string = '\
    created: '+time_now+'\n\
    creator: '+creator+'\n\
    modified: '+time_now+'\n\
    modifier: '+creator+'\n\
    tags: '+image_tags+'\n\
    title: '+tiddler_title+'\n\
    caption: '+image_caption+'\n\
    file_date: '+file_date+'\n\
    local_file: '+whole_file_path+'\n\
    orientation: '+orientation+'\n\
    rotate: '+rotate+'\n\
    img_width: '+width+'\n\
    img_height: '+height+'\n\
    file_type: '+ftype+' \n\
    text: '+embed_text
    workfile = tiddler_title+'.tid'
    filepath = tid_subdir_path / workfile # p is a Path object
    with filepath.open("w", encoding ="utf-8") as f: # I use with, the file gets closed automatically when done.
        f.write(tiddler_string)
    return

In [11]:
def append_tiddler_dict():
    tiddler_dict = {
        "created": time_now,
        "creator": creator,
        "modified": time_now,
        "modifier": creator,
        "tags": image_tags,
        "title": tiddler_title,
        "caption": image_caption,
        "file_date": file_date,
        "local_file": whole_file_path,
        "orientation": orientation,
        "rotate": rotate,
        "img_width": width,
        "img_height": height,
        "file_type": ftype,
        "text": embed_text
        }
    multiple_tiddlers.append(doubleQuoteDict(tiddler_dict)) # see class def for doubleQuoteDict above
    return

## 1) Get the tag's id

In [12]:
# Initialize some things
the_disk = "/data"
the_tag = "birds", # Note the comma, which makes this a tuple
image_ids = [] 

In [13]:
hit = curs.execute("SELECT id from Tags WHERE name == ?", the_tag).fetchone()
if not hit:
    print("no tag called "+ the_tag + " found")
else:
    the_tag_id = hit
    
print(the_tag_id)

(25,)


## 2) Get all `imageid`s matching the tag's `tagid` from the ImageTags table.

In [14]:
image_ids = []
hits = curs.execute("SELECT imageid from ImageTags WHERE tagid == ?", the_tag_id )
for row in hits:
    image_ids.append(row)
# output is a list of singleton tuples 

## Step 3) will loop through the image ids and get info from the database about that image, then compose a tiddler for each. 

Then it will write either a lot of .tid files or one.json file containing the tiddlers.

In [20]:
from datetime import datetime 

multiple_tiddlers = []

p = pathlib.Path("./")
tid_subdir_name = 'tiddlers'+str(datetime.utcnow().strftime('%Y%m%d%H%M%S%f'))
tid_subdir_path = p / tid_subdir_name
tid_subdir_path.mkdir(exist_ok=True)

# Some choices for TiddlyWiki

creator = 'chris'
modifier = 'chris'

# Some filetypes I may run into -- NOTE the raw files should be handled a little differently because they
# can't be previewed as images in the browser -- but so far I haven't done anything
ftypes = {'bmp':'image/bmp', 'jpg':'image/jpeg', 'gif':'image/gif',\
          'png':'image/png', 'avi':'video/x-msvideo', 'mpeg':'video/mpeg', \
          'mp4':'video/mp4', 'raw-rw2':'image/x-panasonic-rw2', \
          'raw-nef':'image/x-nikon-nef', 'raw-crw':'image/x-canon-crw'}

# Corrections needed according to the orientations in the ImageInformation table (some of the zeros are guesses)
rots = {'None':'0', '0':'0', '1':'0', '3':'180', '6':'90', '8':'270'}

for imid in image_ids:
    # Get the current time in UTC, in the format TiddlyWiki uses for dates
    time_now = str(datetime.utcnow().strftime('%Y%m%d%H%M%S%f')[:-3])
    # Get fields for tiddlers using database data
    (f_name, whole_file_path) = get_file_path(imid)
    (orientation, width, height, file_type) = get_image_info(imid)
    file_date = get_file_date(imid)
    image_caption = get_caption(imid)
    rotate = rots[orientation]
    image_tags = get_tags(imid)

    first8chars = file_date[:8]
    tiddler_title = first8chars+f_name
    
    ftype = ftypes[file_type.lower()] # referring to the dict I made earlier
    # Should be tidy and make a function for this too
    if ftype[:5] == 'video':
        embed_text = '<<vid>>'
    elif ftype[:5] == 'image':
        embed_text = '<<dkampic>>'
    else:
        embed_text = 'Didn\'t find a video or image filetype...'

    # If I want individual .tid files for composing a TW using node.js:
    output_as_tid_file()
    
    ## If making one json file:
    #append_tiddler_dict()

# Done with the database
conn.close()


###==== If putting all into one json file =====
# # Construct a filename to write into:
# workfile = 'tiddlers'+time_now+'.json'
# filepath = p / workfile

# # If I use with, the file gets closed automatically when done.
# with filepath.open("w", encoding ="utf-8") as f: 
#     f.write(str(multiple_tiddlers))

